In [18]:
%load_ext autoreload
%autoreload 2

from jenga.tasks.openml import OpenMLTask
from jenga.cleaning.cleaners import AutoGluonCleaner, SklearnCleaner
from jenga.corruptions.numerical import Scaling
from jenga.corruptions.generic import MissingValues
from jenga.evaluation.autocleaning import AutoCleaningEvaluator
from jenga.cleaning.cleaners import PyODKNNSingleColumnCleaner, PyODKNNMultiColumnCleaner
import numpy as np
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
task = OpenMLTask(42261)
model = task.fit_baseline_model(task.train_data, task.train_labels)

Data pickle file already exists and is up to date.


Found 0 categorical columns: []
Found 39 numeric columns: ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'z', 'aa', 'ab', 'ac', 'ad', 'ae', 'af', 'ag', 'ah', 'ai', 'aj', 'ak', 'al', 'am', 'an', 'ao', 'ap', 'ar']
Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    1.6s finished


In [25]:
corruptions = []
random_cols = random.sample(task.numerical_columns + \
                             task.categorical_columns, 5)
for impacted_column in random_cols:
    for fraction in [0.99, 0.5, 0.1]:
        if impacted_column in task.numerical_columns:
            corruptions.append(Scaling(impacted_column, fraction))
        for missingness in ['MCAR', 'MAR', 'MNAR']:
            corruption = MissingValues(impacted_column, fraction, missingness=missingness, na_value=np.nan)
            corruptions.append(corruption)

In [34]:
cleaners = [
    PyODKNNSingleColumnCleaner(numeric_columns=task.numerical_columns,
                               categorical_columns=task.categorical_columns),
    PyODKNNMultiColumnCleaner(numeric_columns=task.numerical_columns,
                               categorical_columns=task.categorical_columns)
]

results = []
for c in cleaners:
    c.fit(task.train_data)
    evaluator = AutoCleaningEvaluator(task, c)
    results.append(evaluator.evaluate(model, 2, *corruptions))


Evaluation 0/120 (0.17411899999999036s)
Evaluation 10/120 (2.042488999999989s)
Evaluation 20/120 (3.896948999999978s)
Evaluation 30/120 (5.67227699999998s)
Evaluation 40/120 (7.4700579999999945s)
Evaluation 50/120 (9.190502999999978s)
Evaluation 60/120 (10.880720999999994s)
Evaluation 70/120 (12.739926999999994s)
Evaluation 80/120 (14.561299999999989s)
Evaluation 90/120 (16.536977999999976s)
Evaluation 100/120 (18.426736999999974s)
Evaluation 110/120 (20.338844999999992s)
Evaluation 0/120 (0.016889999999989413s)
Evaluation 10/120 (0.18488499999997998s)
Evaluation 20/120 (0.4082129999999893s)
Evaluation 30/120 (0.5895119999999849s)
Evaluation 40/120 (0.7591859999999997s)
Evaluation 50/120 (0.9150789999999915s)
Evaluation 60/120 (1.0774849999999958s)
Evaluation 70/120 (1.2346849999999847s)
Evaluation 80/120 (1.397559000000001s)
Evaluation 90/120 (1.5570769999999925s)
Evaluation 100/120 (1.722894999999994s)
Evaluation 110/120 (1.883090999999979s)
